<a href="https://colab.research.google.com/github/savindumahasen/AI-Detection-System-/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# install the necessary libraries
!pip install pandas
!pip install numpy
!pip install opendatasets

In [5]:
# import necessary libraries
import pandas as pd
import numpy as np
import opendatasets as od


In [12]:
## download the dataset
od.download("https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: savinduruhunuhewa
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text


100%|██████████| 350M/350M [00:00<00:00, 629MB/s]

In [10]:
## define the dataset path
dataset="/content/ai-vs-human-text/AI_Human.csv"